In [1]:
import torch
import torchvision
from utils.utils import read_image

In [2]:
image = read_image('./test/spider.png')
model = torchvision.models.vgg16_bn(pretrained=True)
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [3]:
t = model.features[0](image)

In [4]:
t_norm = model.features[1](t)

In [5]:
t_norm[0][0]

tensor([[-0.0407, -0.0524, -0.0564,  ..., -0.1495, -0.1487, -0.1460],
        [-0.0266, -0.0519, -0.0552,  ..., -0.1714, -0.1709, -0.1706],
        [-0.0352, -0.0621, -0.0658,  ..., -0.1720, -0.1704, -0.1699],
        ...,
        [-0.0270, -0.0577, -0.0561,  ...,  0.0520,  0.0562,  0.1059],
        [-0.0269, -0.0571, -0.0539,  ...,  0.0509,  0.0548,  0.1045],
        [ 0.0183,  0.0140,  0.0183,  ...,  0.1914,  0.1958,  0.1710]],
       grad_fn=<SelectBackward>)

In [18]:
stats = model.features[1].state_dict()

In [7]:
stats['weight'], stats['running_var'] = torch.sqrt(stats['running_var']), stats['weight'] ** 2
stats['bias'], stats['running_mean'] = stats['running_mean'], stats['bias']

In [8]:
new_layer = torch.nn.BatchNorm2d(64)
new_layer.load_state_dict(stats)

<All keys matched successfully>

In [19]:
t_norm_renorm = ((t_norm - stats['bias'].view(1, -1, 1, 1)) * (torch.sqrt(stats['running_var'].view(1, -1, 1, 1)+1e-05))/(stats['weight'].view(1, -1, 1, 1)+1e-05)) + stats['running_mean'].view(1, -1, 1, 1)

In [10]:
t_norm_renorm = new_layer(t_norm)

In [20]:
t_norm_renorm[0][0]

tensor([[ 0.1347,  0.1031,  0.0922,  ..., -0.1591, -0.1570, -0.1495],
        [ 0.1727,  0.1043,  0.0954,  ..., -0.2181, -0.2168, -0.2161],
        [ 0.1494,  0.0768,  0.0668,  ..., -0.2197, -0.2154, -0.2140],
        ...,
        [ 0.1714,  0.0886,  0.0929,  ...,  0.3847,  0.3960,  0.5301],
        [ 0.1717,  0.0904,  0.0988,  ...,  0.3818,  0.3924,  0.5264],
        [ 0.2938,  0.2822,  0.2937,  ...,  0.7610,  0.7728,  0.7059]],
       grad_fn=<SelectBackward>)

In [21]:
t[0][0]

tensor([[ 0.1347,  0.1031,  0.0922,  ..., -0.1591, -0.1570, -0.1496],
        [ 0.1727,  0.1043,  0.0954,  ..., -0.2181, -0.2168, -0.2161],
        [ 0.1495,  0.0768,  0.0668,  ..., -0.2197, -0.2154, -0.2140],
        ...,
        [ 0.1715,  0.0886,  0.0929,  ...,  0.3848,  0.3960,  0.5302],
        [ 0.1717,  0.0904,  0.0988,  ...,  0.3818,  0.3924,  0.5264],
        [ 0.2938,  0.2822,  0.2937,  ...,  0.7610,  0.7728,  0.7060]],
       grad_fn=<SelectBackward>)

In [14]:
model.features[28].state_dict()['weight'].shape

torch.Size([512])